In [ ]:
from datetime import datetime

from loguru import logger
from questra_authentication import QuestraAuthentication

from questra_data import QuestraData, QuestraDataCore

url = "https://dev.questra.s2o.dev/dynamic-objects/graphql"
auth_client = QuestraAuthentication(
    url="https://authentik.dev.questra.s2o.dev", interactive=True
)
# questra_data = QuestraData(
# 		graphql_url="https://dev.questra.s2o.dev/dynamic-objects/graphql",
# 		auth_client=auth_client)
client = QuestraData(graphql_url=url, auth_client=auth_client)
core_client = QuestraDataCore(graphql_url=url, auth_client=auth_client)
logger.remove()
# logger.add(
#     sys.stdout,
#     format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level> {extra}",
#     level="INFO",
#     colorize=True,
# )

In [ ]:
items = client.list_items(inventory_name="AllTypTest", properties=["_id","integertyp"],
                  where={"integertyp": {"gt": "50"}})
for item in items:
	print(f"{item['_id']}: {item['integertyp']}")
	#print(item)


In [ ]:
namespace_name = "Trading"
inventory_name = "PowerProducts"
# client.delete_inventory(namespace_name="Trading", inventory_name="PowerProducts")
client.list_quotation_timestamps(
    namespace_name=namespace_name,
    inventory_name=inventory_name,
    timeseries_property="quarter_hourly_prices",
    from_time=datetime(2025, 1, 1),
    to_time=datetime(2025, 3, 31, 23, 59),
)
# client.list_inventories_df(namespace_name=namespace_name, inventory_names=[inventory_name], explode_properties=True)

In [ ]:
# Type of the _id field in the DataFrame is str
client.list_items(
    inventory_name="Hashtags", namespace_name="SocialMedia", limit=50, properties=["tag","posts.title"]
).to_df()

In [ ]:
from questra_data.models.timeseries import TimeSeriesValue

# schreibe auf 1 Zeitreihe
client.save_timeseries_values(
    inventory_name="Sensoren",
    namespace_name="TestDaten",
    timeseries_properties="messwerte_temperatur",
    item_values={
        "638301262349418496": list(
            [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=22.5),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=23.0),
            ]
        ),
        "638301262349418497": list(
            [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=21.7),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=22.1),
            ]
        ),
    },
)

# schreibe auf mehrere Zeitreihen
client.save_timeseries_values(
    inventory_name="Sensoren",
    namespace_name="TestDaten",
    timeseries_properties=["messwerte_temperatur", "messwerte_luftfeuchtigkeit"],
    item_values={
        "638301262349418496": {
            "messwerte_temperatur": [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=22.5),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=23.0),
            ],
            "messwerte_luftfeuchtigkeit": [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=49.3),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=50.2),
            ],
        },
        "638301262349418497": {
            "messwerte_temperatur": [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=21.7),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=22.1),
            ],
            "messwerte_luftfeuchtigkeit": [
                TimeSeriesValue(time=datetime(2024, 6, 20, 12, 0, 0), value=51.7),
                TimeSeriesValue(time=datetime(2024, 6, 21, 12, 0, 0), value=52.0),
            ],
        },
    },
)

In [ ]:
# client.list_inventories_df("AllTypesDemo", explode_properties=True)
client.list_inventories("SocialMedia")

client.list_items(
    "AllDataTypes",
    namespace_name="AllTypesDemo",
    properties=[
        "text_field",
        "int_field",
        "long_field",
        "decimal_field",
        "boolean_field",
        "datetime_field",
        "datetimeoffset_field",
        "date_field",
        "time_field",
        "guid_field",
        "file_field.id",
    ],
).to_df()

In [ ]:
client.create_items(
    inventory_name="AllDataTypes",
    namespace_name="AllTypesDemo",
    items=[{"unique_identifier": "too_long", "decimal_field": "6.253015961"}],
)

In [ ]:
# client.list_inventories(namespace_name="TestDaten")

# client.list_items(inventory_name="Gebaeude", namespace_name="TestDaten", properties=[
#   "gebaeudename", "standort", "raeume.raumnummer", "raeume.typ", "raeume.flaeche_m2", "raeume.sensoren.sensornummer", "raeume.sensoren.typ"]).to_df()

# client.list_items(inventory_name="Raeume", namespace_name="TestDaten", properties=[
#   "gebaeude.gebaeudename", "gebaeude.standort", "raumnummer", "typ", "flaeche_m2", "sensoren.sensornummer", "sensoren.typ"]).to_df()

client.list_items(
    inventory_name="Sensoren",
    namespace_name="TestDaten",
    properties=[
        "raum.gebaeude.gebaeudename",
        "raum.gebaeude.standort",
        "raum.raumnummer",
        "raum.typ",
        "raum.flaeche_m2",
        "sensornummer",
        "typ",
    ],
).to_df()

In [ ]:
# df_single = client.list_timeseries_values_df(
#     inventory_name="Sensoren",
#     namespace_name="TestDaten",
#     timeseries_properties="messwerte_temperatur",
#     properties=["raum.gebaeude.gebaeudename", "raum.raumnummer", "sensornummer", "messwerte_temperatur.timeZone"],
#     from_time=datetime(2024, 1, 1),
#     to_time=datetime(2024, 1, 2)
# )
# df_single
# print(df_single.head(10))

# Mehrere Properties mit Einheiten - PIVOT-FORMAT mit MultiIndex
df_multi = client.list_timeseries_values(
    inventory_name="Sensoren",
    namespace_name="TestDaten",
    timeseries_properties=[
        "messwerte_temperatur",
        "messwerte_luftfeuchtigkeit",
        "messwerte_co2",
    ],
    properties=["sensornummer"],
    from_time=datetime(2024, 1, 1),
    to_time=datetime(2024, 1, 2),
    where={"sensornummer": {"in": ["SENS-0001", "SENS-0004"]}},
).to_df()

df_multi